In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [3]:
from sklearn.model_selection import KFold

In [4]:
from tensorflow.keras.layers import Conv2D

In [5]:
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import concatenate

In [6]:
from tensorflow import keras

In [7]:
from tensorflow.keras.layers import Bidirectional

In [8]:
from tensorflow.keras.models import Model

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation

In [13]:
from tensorflow.python.keras.layers.embeddings import Embedding

In [14]:
import nltk
import string
from nltk.corpus import stopwords
import re
from nltk.stem.snowball import SnowballStemmer

In [38]:
train_df=pd.read_csv('train.csv')

In [39]:
test_df=pd.read_csv('test.csv')

In [40]:
sample = pd.read_csv("sample_submission.csv",encoding = "ISO-8859-1")

In [41]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    ## Convert words to lower case and split them
    text = text.lower()
       
    ## Remove stop words (mejor las dejo)
    #text = text.split()
    #stops = set(stopwords.words("english"))
    #text = [w for w in text if not w in stops and len(w) >= 3]  
    #text = " ".join(text)

    ## Clean the text
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  #quito caracteres inusuales
    text = re.sub('â' , '', text)  #quito caracteres inusuales
    text = re.sub('ª' , '', text)  #quito caracteres inusuales
    text = re.sub('ã' , '', text)  #quito caracteres inusuales
    text = re.sub('¼' , '', text)  #quito caracteres inusuales
    text = re.sub('\w*\d\w*', '', text)  # quito numeros
    text = re.sub("[^0-9a-zA-Z]+", " ", text)
    
    ## Stemming
    #text = text.split()
    #stemmer = SnowballStemmer('english')
    #stemmed_words = [stemmer.stem(word) for word in text]
    #text = " ".join(stemmed_words)

    return text

In [42]:
train_df['text'] = train_df['text'].replace(r'http\S+', '', regex=True)
test_df['text'] = test_df['text'].replace(r'http\S+', '', regex=True)

In [45]:
train_df.text=train_df.text.map(lambda x: clean_text(x))
test_df.text=test_df.text.map(lambda x: clean_text(x))

In [46]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['text'])

train_sequences = tokenizer.texts_to_sequences(train_df['text'])
train_data = pad_sequences(train_sequences, maxlen=180)

In [49]:
test_sequences = tokenizer.texts_to_sequences(test_df['text'])
test_data = pad_sequences(test_sequences, maxlen=180)

In [75]:
EMBEDDING_FILE = 'glove-dataset/glove.twitter.27B.200d.txt' 
#Por ahora lo dejoe n 50, supongo que solo mejora con 200 si complejizo la red nueronal
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, 'r', encoding='utf-8'))

In [79]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((50000, 200)) #50 o 200, segun las dimensiones del objeto que use
for word, index in tokenizer.word_index.items():
    if index > 50000 - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [51]:
model_glove = Sequential()
model_glove.add(Embedding(50000, 50, input_length=180, weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(LSTM(300))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [52]:
#separo train y test
x_train, x_test, y_train, y_test = train_test_split(train_data, train_df['target'], test_size=0.2)

In [53]:
model_glove.fit(x_train, y_train, epochs = 2)

Epoch 1/2
6090/6090 [==============================] - 202s 33ms/sample - loss: 0.4930 - acc: 0.7709
Epoch 2/2
6090/6090 [==============================] - 224s 37ms/sample - loss: 0.4420 - acc: 0.8007


In [54]:
pred = model_glove.predict_classes(x_test)

In [55]:
pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [0],
       [1]])

In [56]:
f1_score(y_test, pred, average='macro')

0.7878148108455685

 # Calculo con k-folds

In [64]:
kf = KFold(n_splits = 5)
kf

KFold(n_splits=5, random_state=None, shuffle=False)

In [125]:
#Igual esta "mal", deberia vectorizar y entrenar solo los valores del train (creo)

In [81]:
test_error=[]
#Si agregamdp una dense mejora el resultado
for train_index,test_index in kf.split(train_data):
    x_train,x_test = train_data[train_index], train_data[test_index]
    y_train,y_test = train_df['target'][train_index], train_df['target'][test_index]
    
    model_glove = Sequential()
    model_glove.add(Embedding(50000, 50, input_length=180, weights=[embedding_matrix], trainable=False))
    model_glove.add(Dropout(0.2))
    model_glove.add(Conv1D(64, 5, activation='relu'))
    model_glove.add(MaxPooling1D(pool_size=4))
    model_glove.add(LSTM(300))
    model_glove.add(Dense(1, activation='sigmoid'))
    model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model_glove.fit(x_train,y_train, epochs=2, batch_size=20)
    
    pred = model_glove.predict_classes(x_test)

    test_error.append(f1_score(y_test, pred, average='macro'))
    
print('Error de test:',test_error)

KeyboardInterrupt: 

# Realizo la prediccion

In [61]:
#Creo el modelo devuelta
model_glove = Sequential()
model_glove.add(Embedding(50000, 50, input_length=100, weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(LSTM(300))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [62]:
model_glove.fit(train_data, train_df['target'], epochs = 2)

Epoch 1/2
7613/7613 [==============================] - 135s 18ms/sample - loss: 0.4923 - acc: 0.7641
Epoch 2/2
7613/7613 [==============================] - 146s 19ms/sample - loss: 0.4462 - acc: 0.7993


In [58]:
test_df['text'] = test_df['text'].map(lambda x: clean_text(x))

In [63]:
test_sequences = tokenizer.texts_to_sequences(test_df['text'])
test_data = pad_sequences(test_sequences, maxlen=180)

In [66]:
predictions= model_glove.predict_classes(test_data)

In [68]:
predictions

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [69]:
sample.target = predictions

In [71]:
sample.to_csv('neural_network_glove_2_epochs.csv', index=False)

In [72]:
sample.target.value_counts()

0    1976
1    1287
Name: target, dtype: int64

# Intento agregarle multiple inputs

In [57]:
from sklearn.preprocessing import MinMaxScaler

In [58]:
cs = MinMaxScaler()

In [59]:
train_input_2 = pd.read_csv("train_limpio_input_2_NN.csv",encoding = "ISO-8859-1")
test_input_2 = pd.read_csv("test_limpio_input_2_NN.csv",encoding = "ISO-8859-1")

In [60]:
#Le saco el target 
target = train_input_2.target
train_input_2 = train_input_2.drop(columns=['target'])

In [61]:
test_input_2 = test_input_2.drop(columns=['text'])
train_input_2 = train_input_2.drop(columns=['text'])

In [62]:
trainContinuous = cs.fit_transform(train_input_2)
testContinuous = cs.fit_transform(test_input_2)

In [63]:
trainContinuous.shape

(7613, 232)

# Contolo con k-folds

In [65]:
nlp_input = keras.Input(shape=(180,), name='nlp_input')
meta_input = keras.Input(shape=(232,), name='meta_input')

In [85]:
test_error=[]
#Si agregamdp una dense mejora el resultado
for train_index,test_index in kf.split(train_data):
    x_train_words,x_test_words = train_data[train_index], train_data[test_index]
    x_train_numerical,x_test_numerical = trainContinuous[train_index], trainContinuous[test_index]
    y_train,y_test = train_df['target'][train_index], train_df['target'][test_index]
    
    emb = Embedding(50000, 200, input_length=100, weights=[embedding_matrix], trainable=False)(nlp_input)
    drop = Dropout(0.2)(emb)#agregada propia (creo que andaba mejor antes de agregar esto)
    conv = Conv1D(64, 5, activation='relu') (drop) #agregada propia (creo que andaba mejor antes de agregar esto)
    mp = MaxPooling1D(pool_size=4)(conv)#agregada propia (creo que andaba mejor antes de agregar esto)
    nlp_out = Bidirectional(LSTM(128))(emb)  #conv antes era emb
    
    x = concatenate([nlp_out, meta_input])
    x = Dense(24, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)
    
    
    model = Model(inputs=[nlp_input , meta_input], outputs=[x])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    
    model.fit([x_train_words, x_train_numerical], y_train, batch_size=128, epochs=5)
    
    predictions= model.predict([x_test_words,x_test_numerical])
    
    bin_pred = []
    for i in predictions:
        if (i<0.5):
            bin_pred.append(0)
        else:
            bin_pred.append(1)

    test_error.append(f1_score(y_test, bin_pred, average='macro'))
    
print('Error de test:',test_error)

Epoch 1/5
 256/6090 [>.............................] - ETA: 14:54 - loss: 0.6670 - acc: 0.6289

KeyboardInterrupt: 

In [ ]:
#Con dropout, conv1D, maxpooling y 3 epochs (estandar):
Error de test: [0.7888178425057502, 0.7914766253113024, 0.7840449223167527, 0.746530115508615, 0.7930096559227526]
    
#SIN dropout,ni conv1D,ni maxpooling y con 3 epochs (da un poquito mejor sin nada de lo que le agregue):
Error de test: [0.7907451626963822, 0.786468158944994, 0.7854915546911962, 0.7873662261730481, 0.8080302711793146]
    
#Con dropout, conv1D, maxpooling PERO con 5 epochs (mejoro un poquito tambien):
Error de test: [0.805207329733444, 0.7851134723292488, 0.7747011121720304, 0.7838792991204754, 0.793572611501342]
    
#Con dropout, conv1D, maxpooling y 3 epochs PERO con vectores de 200dims:
Error de test: [0.8013598421267523, 0.7831023324122048, 0.7737440449504573, 0.7866347906188544, 0.8026946909920577]

#Mezcla de todo, ya que con todo lo mejoró(tarda una banda):



In [37]:
trainContinuous.shape

(7613, 232)

In [86]:
nlp_input = keras.Input(shape=(180,), name='nlp_input')
meta_input = keras.Input(shape=(232,), name='meta_input')

In [87]:
emb = Embedding(50000, 200, input_length=100, weights=[embedding_matrix], trainable=False)(nlp_input)
#50000, 50, input_length=100, weights=[embedding_matrix], trainable=False

In [138]:
drop = Dropout(0.2)(emb)#agregada propia (creo que andaba mejor antes de agregar esto)

In [139]:
conv = Conv1D(64, 5, activation='relu') (drop) #agregada propia (creo que andaba mejor antes de agregar esto)
#x = Conv2D(f, (3, 3), padding="same")(x)

In [140]:
mp = MaxPooling1D(pool_size=4)(conv)#agregada propia (creo que andaba mejor antes de agregar esto)

In [88]:
nlp_out = Bidirectional(LSTM(128))(emb)  #conv antes era emb

In [89]:
x = concatenate([nlp_out, meta_input])

In [90]:
x = Dense(24, activation='relu')(x)

In [91]:
x = Dense(1, activation='sigmoid')(x)

In [92]:
model = Model(inputs=[nlp_input , meta_input], outputs=[x])

In [93]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [94]:
model.summary()

Model: "model_23"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
nlp_input (InputLayer)          [(None, 180)]        0                                            
__________________________________________________________________________________________________
embedding_27 (Embedding)        (None, 180, 200)     10000000    nlp_input[0][0]                  
__________________________________________________________________________________________________
bidirectional_23 (Bidirectional (None, 256)          336896      embedding_27[0][0]               
__________________________________________________________________________________________________
meta_input (InputLayer)         [(None, 232)]        0                                            
___________________________________________________________________________________________

In [ ]:
model.fit([train_data, trainContinuous], target, batch_size=32, epochs=5)

Epoch 1/5
 448/7613 [>.............................] - ETA: 15:11 - loss: 0.6472 - acc: 0.6585

In [149]:
test_data.shape

(3263, 180)

In [150]:
predictions= model.predict([test_data,testContinuous])

In [151]:
predictions

array([[0.6548752 ],
       [0.8795717 ],
       [0.97370327],
       ...,
       [0.85783964],
       [0.8268718 ],
       [0.5158545 ]], dtype=float32)

In [152]:
bin_pred = []
for i in predictions:
    if (i<0.5):
        bin_pred.append(0)
    else:
        bin_pred.append(1)

In [153]:
sample.target = bin_pred

In [156]:
sample.target.value_counts() #con convD1 pero con epochs=3 en lugar de 2

0    2110
1    1153
Name: target, dtype: int64

In [135]:
sample.target.value_counts() #con convD1

0    2032
1    1231
Name: target, dtype: int64

In [69]:
sample.target.value_counts() #sin convD1

0    2109
1    1154
Name: target, dtype: int64

In [155]:
sample.to_csv('neural_network_doble_input_triple_epochs.csv', index=False)